# prep table1 

In [55]:
%cd /Users/sumishunsuke/Desktop/RNA/genzyme

import os 
import sys
import subprocess
from pprint import pprint 
sys.path.append("src")

import pandas as pd 
from Bio import SeqIO


/Users/sumishunsuke/Desktop/RNA/genzyme


## pred activity by RfamGen

In [22]:
rfams = ["RF00005_CCU", "RF00234", "RF03160"]

for rfam in rfams:
    h5 = !ls ./datasets/ForDMSdata/{rfam}/*cm*test*.h5
    print(h5)

['./datasets/ForDMSdata/RF00005_CCU/RF00005_unique_CCUanticodon_notrunc_traceback_onehot_cm_test.h5', './datasets/ForDMSdata/RF00005_CCU/RF00005_unique_CCUanticodon_notrunc_traceback_onehot_cm_test_weight_threshold0p2.h5']
['./datasets/ForDMSdata/RF00234/RF00234_unique_addtemp_notrunc_traceback_onehot_cm_test.h5', './datasets/ForDMSdata/RF00234/RF00234_unique_addtemp_notrunc_traceback_onehot_cm_test_weight_threshold0p2.h5']
['./datasets/ForDMSdata/RF03160/RF03160_unique_addtemp_AGUCT_notrunc_traceback_onehot_cm_test.h5', './datasets/ForDMSdata/RF03160/RF03160_unique_addtemp_AGUCT_notrunc_traceback_onehot_cm_test_weight_threshold0p2.h5']


### select best model using test data

In [44]:
dict_best = {}

for rfam in rfams:
    h5  = !ls ./datasets/ForDMSdata/{rfam}/*cm*test.h5
    h5 = h5[0]
    h5w = !ls ./datasets/ForDMSdata/{rfam}/*cm*test*weight*.h5
    h5w = h5w[0]

    # load final ckpt
    ckpts = !ls -t ./outputs/ForDMSdata/{rfam}/timecourse_cmvae/*.pt
    ckpts_dict = {f"trial{str(i).zfill(2)}":{} for i in range(10)}
    for ckpt in ckpts:
        ckpt_name = ckpt.split("/")[-1].split("_")[-1].replace(".pt", "")
        epoch = int(ckpt.split("/")[-1].split("_")[-2].replace("epoch", ""))
        ckpts_dict[ckpt_name][epoch] = ckpt
    ckpt_final = {}
    for trial, epoch_dict in ckpts_dict.items():
        ckpt_final[trial] = epoch_dict[max(epoch_dict.keys())]
    # ckpt_final["trial00"]

    besttest = 10000000
    for ckpt in ckpt_final.values():

        cmd = f"""python scripts/eval_CMVAE_by_test.py --X_test {h5} --w_test {h5w} \
            --config ./outputs/ForDMSdata/{rfam}/timecourse_cmvae/config_trial00.yaml \
            --ckpt {ckpt} 
            """
        
        res = subprocess.run(cmd, shell=True,capture_output=True)
        if besttest > float(res.stdout.decode()):
            besttest = float(res.stdout.decode())
            dict_best[rfam] = (ckpt, besttest)
            


In [45]:
pprint(dict_best)

{'RF00005_CCU': ('./outputs/ForDMSdata/RF00005_CCU/timecourse_cmvae/model_epoch60_trial04.pt',
                 2.534673621225204),
 'RF00234': ('./outputs/ForDMSdata/RF00234/timecourse_cmvae/model_epoch79_trial03.pt',
             504.35129182235056),
 'RF03160': ('./outputs/ForDMSdata/RF03160/timecourse_cmvae/model_epoch64_trial07.pt',
             13.140731165231752)}


## load activity data

In [46]:
dict_best

{'RF00005_CCU': ('./outputs/ForDMSdata/RF00005_CCU/timecourse_cmvae/model_epoch60_trial04.pt',
  2.534673621225204),
 'RF00234': ('./outputs/ForDMSdata/RF00234/timecourse_cmvae/model_epoch79_trial03.pt',
  504.35129182235056),
 'RF03160': ('./outputs/ForDMSdata/RF03160/timecourse_cmvae/model_epoch64_trial07.pt',
  13.140731165231752)}

In [71]:
activity = {}

# RF00005_CCU
activity["RF00005_CCU"] = pd.read_table(f"./datasets/ForDMSdata/RF00005_CCU/activity/FitnessData.txt", sep = "\t", skiprows =6)["Fit"].tolist()


# RF00234
df = pd.read_csv("./datasets/ForDMSdata/RF00234/activity/additional_data_andearson_2020/glmS ribozyme RNA array_Source Data/dataframe1_kobs_kcat_KM_rescues.csv")
df = df.dropna(subset = ["kcat"])
df = df[df["MismatchCount"] >0]
activity["RF00234"] = df["kcat"].tolist()


#RF01788
cleavage_data = f"./datasets/ForDMSdata/RF01788/activity/sb7b00367_si_001.xlsx"
df_cleavage = pd.read_excel(cleavage_data)
activity["RF01788"] = df_cleavage["FC"].tolist()


# RF03160
info_mut = []
fasta = open("./datasets/ForDMSdata/RF03160/activity/Kobori2016.fa", "r")
for record in SeqIO.parse(fasta, "fasta"):
    mut = record.description.split(" ")[-1].split("/")
    if len(mut) == 1:
        mut = [mut[0], mut[0]]
    info_mut.append(mut)

df_act = pd.read_excel("./datasets/ForDMSdata/RF03160/activity/Kobori_ACIE_2016_Supporting_Data.xlsx", skiprows=2, index_col = 0)
fitness = []
for mut in info_mut: 
    fitness.append(df_act[mut[0]][mut[1]])
activity["RF03160"] = fitness


activity["RF01788"][:5]

[0.8629249865083648,
 0.8585795097423005,
 0.8523351648351648,
 0.8518760907504364,
 0.8509212730318257]

## pred DMS 

In [58]:
!ls ./datasets/ForDMSdata/*/activity/*cm.h5

./datasets/ForDMSdata/RF00005_CCU/activity/Li2016_notrunc_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF00234/activity/Andreasson2020_notrunc_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF01788/activity/Kobori2018_notrunc_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF01846/activity/E1_Small_1_30C_Glu_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF01846/activity/E2_Small_2_37C_Glu_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF01846/activity/E4_Small_3_30C_Gal_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF01846/activity/E5_Big_1_30C_Glu_traceback_onehot_cm.h5
./datasets/ForDMSdata/RF03160/activity/Kobori2016_notrunc_traceback_onehot_cm.h5


In [64]:
for rfam in ["RF00005_CCU", "RF00234", "RF03160"]:
    dms_trsp = !ls ./datasets/ForDMSdata/{rfam}/activity/*cm.h5
    dms_trsp = dms_trsp[0]

    best_ckpt = dict_best[rfam][0]

    cmd = f"""
    python ./scripts/pred_activity_with_CMVAE.py \
    --in_trsp {dms_trsp} \
    --ckpt {best_ckpt} \
    --model_config ./outputs/ForDMSdata/{rfam}/timecourse_cmvae/config_trial00.yaml \
    --n_samples 5 \
    --output ./outputs/ForDMSdata/{rfam}/timecourse_cmvae/elbo_dms_final_epoch.txt
    """
    res = subprocess.run(cmd, shell = True, capture_output = True)
    print(res.stderr.decode())

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.

[W NNPACK.cpp:80] Could not initialize NNPACK! Reason: Unsupported hardware.



## calc spearman of elbo v.s. dms

In [ ]:
def load_elbo_results(file):
    eve = []
    with open(file, "r") as f: 
        for line in f:
            if not line.startswith("#"):
                eve.append(float(line))

    return eve

dict_elbo = {}
for rfam in ["RF00005_CCU", "RF00234", "RF03160"]:
    file = f"./outputs/ForDMSdata/{rfam}/timecourse_cmvae/elbo_dms_final_epoch.txt"
    dict_elbo[rfam] = load_elbo_results(file)

dict_elbo

In [70]:
from scipy.stats import spearmanr

for rfam in rfams:
    corr, pval = spearmanr(dict_elbo[rfam], activity[rfam])
    print(f"{rfam}:\tcorr:{abs(corr):.4f} (pval: {pval:.4f})")


RF00005_CCU:	corr:0.5562 (pval: 0.0000)
RF00234:	corr:0.5457 (pval: 0.0000)
RF03160:	corr:0.4254 (pval: 0.0000)
